## *Importing required libraries*

In [32]:
!pip install gensim[gensim-4.3.2,cuda]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [33]:
import pandas as pd
import gensim

## *Merged 2 excel sheets datasets, in total we have used 329 asanas benefits*

In [34]:
df = pd.read_csv('test.csv')

In [35]:
df

,Asana,Benefits
0,PADOTTHANASANA,This asana strengthens\nthe abdominal muscles ...
1,PARVATASANA,This pose strengthens the nerves and muscles i...
2,ARDHA TITALI ASANA,This is an excellent \npreparatory practice fo...
3,GATYATMAK MERU \nVAKRASANA,This asana removes stiffness \nof the back and...
4,SIDEWAYS VIEWING,Sideways viewing relaxes the \ntension of the ...
...,...,...
321,kapilasana,"1. Helps in stretching legs, hamstrings, arms,..."
322,omkarasana,It relieves the pain of hands and feet( It is ...
323,kashyapawsana,1. This asana provides a deep muscular massage...
324,bhunamanasana,1. Bhunamanasana stretches and improves the fl...


In [36]:
#shape of the data frame
df.shape

(326, 2)

## *Removing Stop words at first and converting it into lower cases so that all stop words can be remove which were in Upper cases.*


In [37]:
!pip install nltk
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')
#  1st step for removing stop words is to use the library and remove stop words, this will make 
# sure that some basic stop words and numerical values is been removed from the Benefits column.

# 2nd step is that there might be stop words present in upper case for example "This" is a stop words 
# which was not removed in 1st step so I converted the Benefits column to lower case. Moreover we can only  
# lowercase any sentence or entire column when that column is free of any numerical value.

# 3rd then further removing stopwords, in this way we get the whole benefits column free from stop words


from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_multiple_whitespaces
df['Benefits'] = df['Benefits'].apply(str.lower)
df['Benefits']=df.Benefits.apply(remove_stopwords)
df['Benefits']=df.Benefits.apply(strip_non_alphanum)
df['Benefits']=df.Benefits.apply(strip_numeric)
df['Benefits']=df.Benefits.apply(strip_multiple_whitespaces)
df['Asana']=df.Asana.apply(strip_multiple_whitespaces)
df['Asana'] = df['Asana'].apply(str.lower)
df

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\atrij\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Asana,Benefits
0,padotthanasana,asana strengthens abdominal muscles massages o...
1,parvatasana,pose strengthens nerves muscles limbs back hel...
2,ardha titali asana,excellent preparatory practice loosening knee ...
3,gatyatmak meru vakrasana,asana removes stiffness increases flexibility ...
4,sideways viewing,sideways viewing relaxes tension muscles strai...
...,...,...
321,kapilasana,helps stretching legs hamstrings arms chest b...
322,omkarasana,relieves pain hands feet helpful rheumatism gi...
323,kashyapawsana,asana provides deep muscular massage abdomen ...
324,bhunamanasana,bhunamanasana stretches improves flexibility ...


## *We have further removed special characters and tokenized each row of benefits.* 

In [38]:
# this will create the list of each benefit rows
benefits =df['Benefits'].apply(gensim.utils.simple_preprocess)
print(benefits)

0      [asana, strengthens, abdominal, muscles, massa...
1      [pose, strengthens, nerves, muscles, limbs, ba...
2      [excellent, preparatory, practice, loosening, ...
3      [asana, removes, stiffness, increases, flexibi...
4      [sideways, viewing, relaxes, tension, muscles,...
                             ...                        
321    [helps, stretching, legs, hamstrings, arms, ch...
322    [relieves, pain, hands, feet, helpful, rheumat...
323    [asana, provides, deep, muscular, massage, abd...
324    [bhunamanasana, stretches, improves, flexibili...
325    [stretches, strengthens, lengthens, mandalasan...
Name: Benefits, Length: 326, dtype: object


In [39]:
# prints the words present in the benefit row 1
benefits[0]

['asana',
 'strengthens',
 'abdominal',
 'muscles',
 'massages',
 'organs',
 'strengthens',
 'digestive',
 'system',
 'lower',
 'back',
 'pelvic',
 'perineal',
 'muscles',
 'helps',
 'correct',
 'prolapse']

## *Implemented Word2Vec with following features:*
* window size = 5
* minimum word cound for which the model can input row wise data = 2
* required cpu  threads to train the model = 4
* size of the required vector embedding = 50

In [40]:

model = gensim.models.Word2Vec(
    window=5,
    min_count=2,
    workers=4,
    vector_size = 50,
)

# model = gensim.models.Word2Vec(df['Benefits'], workers=4, vector_size=50, window=5, sg=0, min_count=2, epochs=2000)


## *Building vocabulary of unique words present in the entire benefit column*

In [41]:

model.build_vocab(benefits, progress_per=5)
# vocab_len = len(model.wv)
# print(vocab_len)

In [42]:
model.train(benefits, total_examples=model.corpus_count, epochs=2000)

(22225305, 29294000)

## *We can test our model for any words suppose say sciatica we will get the similar words in benefits* 

In [43]:
#when we find the similar words for a disease in 
#Benefits section we also see that we get the names of asanas also.
#like for this word we get similar words like chakrasana, marjariasana, shalabhasana which indicates
#that since we have taken these words from benefits section, it means that for this particular diseaes
#most prpbable asanas can be chakrasana, marjariasana, shalabhasana etc.  to cure it.
model.wv.most_similar("sciatica", topn= 100)

#print(type(model.wv.most_similar("sciatica", topn= 100)))

[('stiff', 0.44562867283821106),
 ('rid', 0.4144158363342285),
 ('mild', 0.4039938747882843),
 ('sinus', 0.38537994027137756),
 ('flexibility', 0.3712855279445648),
 ('rounded', 0.36682355403900146),
 ('getting', 0.3617568016052246),
 ('loss', 0.3569590747356415),
 ('pains', 0.35466334223747253),
 ('inside', 0.35138195753097534),
 ('sciatic', 0.35053014755249023),
 ('sacrum', 0.34977027773857117),
 ('marjariasana', 0.34598231315612793),
 ('carotid', 0.34240004420280457),
 ('certain', 0.33649224042892456),
 ('drooping', 0.3348408639431),
 ('backache', 0.3309808671474457),
 ('sleep', 0.33027881383895874),
 ('problem', 0.3267764747142792),
 ('injured', 0.3229636251926422),
 ('issues', 0.3210863471031189),
 ('wide', 0.31878167390823364),
 ('lumbago', 0.31624269485473633),
 ('disc', 0.31309980154037476),
 ('spondylitis', 0.3057402968406677),
 ('lung', 0.30348557233810425),
 ('time', 0.301672101020813),
 ('mobility', 0.30058255791664124),
 ('lengthening', 0.29361239075660706),
 ('waist', 0.2

## *Printing one of the benefits column word's vector*

In [44]:
# suppose we want to know the vector embedding of a word 'pain' so we have printed the vector embedding of size 50
import numpy as np
print((model.wv.get_vector('pain')))

[ 0.5002922  -3.1087303   2.3507447   1.4037863   4.244342    2.3968534
  7.9570065  -1.1527683  13.184853    2.238894   -6.464646    1.1147499
 -0.15096146  3.8199348  -6.623933    7.030395   -0.9710559  -1.0985711
 -2.1148252  -3.0971236   0.9056374   1.6479533   3.590159    9.397978
 -4.2825713  -2.4834735   4.601016    0.43714926 -3.8477886   0.77198577
 -0.95747674  3.2966747  -0.5117349   1.5727175   5.929541   11.865683
  4.25337    10.2608      4.850485    5.199079    4.326882   -1.4786414
  4.483505    1.1075814  -2.152069    7.9868326  -0.23820423  0.45990035
 -2.8428824   3.2836063 ]


In [45]:
# counts total number of rows in datasets being trained
model.corpus_count

326

In [46]:
from gensim.models import Word2Vec
# created list of unique words from the column Benefits. 
# words = list(w for w in model.wv.vocab)
words = list(model.wv.key_to_index.keys())

In [47]:
# printing unique words from the vocabulary list.
print(words)

['muscles', 'body', 'pose', 'helps', 'organs', 'asana', 'spine', 'improves', 'abdominal', 'hips', 'shoulders', 'strengthens', 'blood', 'practice', 'lower', 'balance', 'stretches', 'stretch', 'tones', 'chest', 'legs', 'flexibility', 'neck', 'good', 'arms', 'posture', 'abdomen', 'nervous', 'mind', 'back', 'flow', 'yoga', 'circulation', 'reproductive', 'pelvic', 'system', 'leg', 'hip', 'entire', 'stimulates', 'awareness', 'nerves', 'spinal', 'core', 'improving', 'deep', 'digestive', 'heart', 'gives', 'digestion', 'joints', 'pressure', 'benefits', 'chakra', 'increases', 'strength', 'concentration', 'great', 'upper', 'energy', 'constipation', 'sense', 'thighs', 'functioning', 'like', 'related', 'toning', 'region', 'glands', 'stretching', 'knees', 'breathing', 'toned', 'especially', 'poses', 'focus', 'hamstrings', 'breath', 'strong', 'massages', 'area', 'liver', 'ankles', 'internal', 'better', 'help', 'improve', 'pain', 'the', 'alignment', 'brain', 'disorders', 'stability', 'flexible', 'leve

In [48]:
# here we have the length of unique words

print(len(words))

1330



## *Created an empty dictionary at first to store the unique words as key words  along with its vector embeddings.*

In [49]:
dict_of_word_embeddings = dict({})
for i in words:
    dict_of_word_embeddings[i] = model.wv[i]

In [50]:
# on printing the dictionary, we get the following result.
print(dict_of_word_embeddings)

{'muscles': array([-0.00394978, -0.3323843 ,  2.8532405 ,  0.55789685, -1.4433213 ,
       -3.64525   ,  0.11791018,  0.39306417,  0.04879813,  2.600258  ,
       -0.80486286, -0.52803534,  0.14891097, -0.509645  , -1.7460333 ,
       -0.0901184 ,  3.7690392 ,  0.74101776, -3.4416118 ,  0.89437574,
        1.6300151 , -1.2324969 , -3.0288746 , -2.4513755 ,  0.65482825,
        0.60934526, -0.5886853 , -2.9459221 , -2.945762  ,  0.89702475,
       -0.47673625,  0.4612681 , -0.2026735 ,  2.322153  , -1.5465754 ,
        1.3427222 , -2.5681086 ,  2.2790053 ,  1.7543256 ,  0.12885743,
        0.14740027, -0.3823001 ,  1.2410132 , -3.8494415 ,  0.28885475,
       -0.7170755 ,  0.57176924,  1.8258325 , -2.0902574 ,  1.4759985 ],
      dtype=float32), 'body': array([ 0.7034037 ,  0.7837889 ,  0.6817323 ,  0.23656261, -0.7154826 ,
       -0.7015188 ,  0.58223855, -1.2387677 ,  1.7193204 , -0.03844704,
       -1.4046338 ,  0.5516204 ,  2.091988  ,  0.27356988, -3.1071846 ,
        0.6477713 ,  

## *Exporting dictionary data into separate excel sheet/ csv file*.

In [51]:
Unique_words = dict_of_word_embeddings.keys()
word_vectors  = dict_of_word_embeddings.values()
# print(asanas, word_vectors)
d = {'Unique_words' : Unique_words , 'Word_Vectors' : word_vectors}
dataframe = pd.DataFrame(data = d)
dataframe

,Unique_words,Word_Vectors
0,muscles,"[-0.003949778, -0.3323843, 2.8532405, 0.557896..."
1,body,"[0.7034037, 0.7837889, 0.6817323, 0.23656261, ..."
2,pose,"[-1.5653111, -1.0234802, 1.3111881, 0.888679, ..."
3,helps,"[-0.32310972, -0.18380019, 0.27075425, -0.2924..."
4,organs,"[0.6840859, 1.0092179, 0.9400223, -3.2952435, ..."
...,...,...
1325,little,"[0.39908996, 4.284521, -2.265193, 2.327588, 3...."
1326,trikonasana,"[0.32798702, 0.84546673, 4.7864685, 0.6937671,..."
1327,migraine,"[-4.807702, 2.4231853, 4.4265, 5.765761, 6.301..."
1328,heals,"[-1.5044563, 1.1594666, 3.3377209, 2.1595109, ..."


## *We have made list of unique asanas as after merging excel sheets there were repeated asanas*

In [52]:
asanas = list(df['Asana'])
# total asanas present ( with repetition)
print(len(asanas))
asana = []
      

for x in asanas:
  if x not in asana:
    asana.append(x)
# total number of unique asanas
print(len(asana))    
# list of unique asanas
print(asana)

326
293
['padotthanasana', 'parvatasana', 'ardha titali asana', 'gatyatmak meru vakrasana', 'sideways viewing', 'makarasana', 'padmasana', 'vajrasana', 'ardha chandrasana', 'yogamudrasana', 'bhujangasana', 'saithalyasana', 'bhu namanasana', 'sarvangasana', 'natarajasana', 'poorna bhujangasana', 'koormasana', 'poorna shalabhasana', 'poorna dhanurasana', 'bandha hasta utthanasana ', 'shava udarakarshanasana ', 'chakki chalanasana ', 'kashtha takshanasana ', 'vayu nishkasana', 'ushtrasana', 'samakonasana ', 'matsyasana', 'kandharasana', ' setu asana ', 'paschimottanasana', 'meru akarshanasana', 'pada hastasana', 'seetkari pranayama', 'jalandhara bandha', 'tadagi mudra', 'maha vedha mudra', 'shashankasana', 'janu chakra', 'poorna titali asana', 'manibandha chakra', 'skandha chakra', 'greeva sanchalana', 'padachakrasana', 'pada sanchalanasana', 'supta pawanmuktasana', 'jhulana lurhakanasana', 'supta udarakarshanasana', 'naukasana', 'rajju karshanasana', 'nauka sanchalanasana', 'namaskarasan

## *On-hot Encodded the asanas words and created a dictionary to store the asana word with its word embedding*

In [53]:
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(asana)
#print(integer_encoded)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)

### One hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

# onehot_encoded

asan_dict={}

for i in range(len(asana)):
  asan_dict[asana[i]] = onehot_encoded[i]

print(asan_dict) 


C:\Users\atrij\anaconda3\envs\tf\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


{'padotthanasana': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
      

In [54]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

In [55]:
# printing the benefit words of one row.
print(benefits[0])

['asana', 'strengthens', 'abdominal', 'muscles', 'massages', 'organs', 'strengthens', 'digestive', 'system', 'lower', 'back', 'pelvic', 'perineal', 'muscles', 'helps', 'correct', 'prolapse']


## *Created a Tupple associating the benefits words of each row with the respective asana. In order to bring the associativity between benefits words and asana words*

In [56]:
pair=[]

i=0
a=len(asana)
for x in benefits:
  if(i<a):
    target=asana[i]
    for y in x:
      if(y not in words):
        continue
      pair.append((y,target))
  i+=1  
print(pair)




[('asana', 'padotthanasana'), ('strengthens', 'padotthanasana'), ('abdominal', 'padotthanasana'), ('muscles', 'padotthanasana'), ('massages', 'padotthanasana'), ('organs', 'padotthanasana'), ('strengthens', 'padotthanasana'), ('digestive', 'padotthanasana'), ('system', 'padotthanasana'), ('lower', 'padotthanasana'), ('back', 'padotthanasana'), ('pelvic', 'padotthanasana'), ('muscles', 'padotthanasana'), ('helps', 'padotthanasana'), ('correct', 'padotthanasana'), ('prolapse', 'padotthanasana'), ('pose', 'parvatasana'), ('strengthens', 'parvatasana'), ('nerves', 'parvatasana'), ('muscles', 'parvatasana'), ('limbs', 'parvatasana'), ('back', 'parvatasana'), ('helps', 'parvatasana'), ('increase', 'parvatasana'), ('height', 'parvatasana'), ('stretching', 'parvatasana'), ('muscles', 'parvatasana'), ('ligaments', 'parvatasana'), ('enabling', 'parvatasana'), ('growing', 'parvatasana'), ('bones', 'parvatasana'), ('grow', 'parvatasana'), ('longer', 'parvatasana'), ('circulation', 'parvatasana'), 

## *Making a 2-D array of context words(benefit words) and Target words(asana words) by numpy stack*

In [57]:
contexts=[dict_of_word_embeddings[context] for context,target in pair]
contexts=np.vstack(contexts)
# shape of the context words matrix
contexts.shape


(12085, 50)

In [58]:
targets=[asan_dict[target] for context,target in pair]
targets=np.vstack(targets)
# shape of the target words matrix

targets.shape

(12085, 293)

## *Implemented **Artificial Neural Network**. with the help of tensorfow's keras functional API with the following features:*

* Number of input layer = 1
* Size of input layer = 50 unit
* Number of dense layer = 1
* size of dense layer = 1000 units
* Number of output layer = 1
* size of output unit = 293
* activation function = softmax
* loss = categorical_crossentropy
* optimizer = adam
* number of epotchs = 100

In [59]:
from tensorflow import keras
from tensorflow.keras.layers import Dense

network_input = keras.Input(shape=contexts.shape[1], name='input_layer')
                                                                                    # Create a hidden layer for the network; store under 'hidden_layer'
hidden_layer1 = Dense(units=1000, activation='relu', name='hidden_layer1')(network_input)


hidden_layer2 = Dense(units=1000, activation='relu', name='hidden_layer2')(hidden_layer1)


hidden_layer3 = Dense(units=500, activation='relu', name='hidden_layer3')(hidden_layer2)

                                                                                            # Create an output layer for the network; store under 'output_layer'
output_layer = Dense(units=targets.shape[1], activation='softmax', name='output_layer')(hidden_layer3)

                                                                          # Create a Keras Model; store under 'embedding_model'
embedding_model = keras.Model(inputs=network_input, outputs=output_layer)

                                                          # Compile the model for training; define loss function
embedding_model.compile(loss='categorical_crossentropy',optimizer = 'adam', metrics = ['accuracy'])

                                                          # Print out a summary of the model
embedding_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 50)]              0         
                                                                 
 hidden_layer1 (Dense)       (None, 1000)              51000     
                                                                 
 hidden_layer2 (Dense)       (None, 1000)              1001000   
                                                                 
 hidden_layer3 (Dense)       (None, 500)               500500    
                                                                 
 output_layer (Dense)        (None, 293)               146793    
                                                                 
Total params: 1,699,293
Trainable params: 1,699,293
Non-trainable params: 0
_________________________________________________________________


In [60]:
# Fit a model to the data
embedding_model.fit(x=contexts,   # inputs
                    y=targets,   # outputs
                    batch_size=1024,  # how many pairs of words processed simultaneously
                    epochs=100,   # how many times we loop through the whole data
                    verbose=1   # do not print training status
                   )



Epoch 1/100
12/12 [==============================] - 1s 16ms/step - loss: 5.3523 - accuracy: 0.0279
Epoch 2/100
12/12 [==============================] - 0s 7ms/step - loss: 4.7847 - accuracy: 0.0774
Epoch 3/100
12/12 [==============================] - 0s 7ms/step - loss: 4.4371 - accuracy: 0.1010
Epoch 4/100
12/12 [==============================] - 0s 6ms/step - loss: 4.1373 - accuracy: 0.1184
Epoch 5/100
12/12 [==============================] - 0s 6ms/step - loss: 3.9097 - accuracy: 0.1293
Epoch 6/100
12/12 [==============================] - 0s 6ms/step - loss: 3.7462 - accuracy: 0.1328
Epoch 7/100
12/12 [==============================] - 0s 6ms/step - loss: 3.6198 - accuracy: 0.1314
Epoch 8/100
12/12 [==============================] - 0s 6ms/step - loss: 3.5159 - accuracy: 0.1298
Epoch 9/100
12/12 [==============================] - 0s 6ms/step - loss: 3.4357 - accuracy: 0.1355
Epoch 10/100
12/12 [==============================] - 0s 6ms/step - loss: 3.3890 - accuracy: 0.1326
Epoch 11

12/12 [==============================] - 0s 6ms/step - loss: 2.9647 - accuracy: 0.1327
Epoch 84/100
12/12 [==============================] - 0s 7ms/step - loss: 2.9607 - accuracy: 0.1297
Epoch 85/100
12/12 [==============================] - 0s 7ms/step - loss: 2.9604 - accuracy: 0.1339
Epoch 86/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9629 - accuracy: 0.1309
Epoch 87/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9613 - accuracy: 0.1316
Epoch 88/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9578 - accuracy: 0.1340
Epoch 89/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9587 - accuracy: 0.1351
Epoch 90/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9549 - accuracy: 0.1310
Epoch 91/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9549 - accuracy: 0.1326
Epoch 92/100
12/12 [==============================] - 0s 6ms/step - loss: 2.9605 - accuracy: 0.1291
Epoch 93/100


## *Function which inputs the user details and suggests user with the most recommended asanas*

In [ ]:
from collections import Counter
from IPython.display import clear_output

def magic():
  predicted_asanas = []
  user_input_words= []
  final_predicted_asanas = []
  number_in_words = ['first','second', 'third', 'fourth']
#   number_in_words = ['first']
  for i in range(4):
    user_input_words.append(input(f"Enter {number_in_words[i]} benefit word:  "))
  for i in user_input_words:
    if i in dict_of_word_embeddings:

      input_array = np.expand_dims(dict_of_word_embeddings[i], axis=0)
      prediction = embedding_model.predict(input_array)
      flatten_pred = prediction.flatten()
      result_indices = flatten_pred.argsort()[-10:][::-1]
    
      for result in result_indices:
        predicted_asanas.append(asana[result])
    
    
  counter_found = Counter(predicted_asanas)
  final_predicted_asanas_with_freq = counter_found.most_common(7)
  # print(final_predicted_asanas_with_freq)

  for yoga, freq in final_predicted_asanas_with_freq:
    final_predicted_asanas.append(yoga)
  
  print(final_predicted_asanas)
  choice=input("Clear output: Y/N ")
  if choice=='Y':
   clear_output()
  
  
magic()

Enter first benefit word:  Shoulder
Enter second benefit word:  Joint
Enter third benefit word:  Pain
Enter fourth benefit word:  stiff
1/1 [==============================] - 0s 17ms/step
['bhadrasana', 'kati chakrasana', 'kraunchasana', 'parivritta parsvakonasana', 'bhekasana', ' sirshapada bhumi sparshasana', ' niralamba sirshasana']
